In [2]:
import cv2
import os
import PIL
import tensorflow_hub as hub

In [3]:
data_dir =r"F:\project\COVID-19_Radiography_Dataset\data"

In [4]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

WindowsPath('F:/project/COVID-19_Radiography_Dataset/data')

In [6]:
import random 
COVID = list(data_dir.glob('COVID/*'))
COVID = random.sample(COVID,1000)
len(COVID)

1000

In [7]:
Lung_Opacity= list(data_dir.glob('Lung_Opacity/*'))
Lung_Opacity = random.sample(Lung_Opacity,1000)
len(Lung_Opacity)

1000

In [8]:
Normal = list(data_dir.glob('Normal/*'))
Normal = random.sample(Normal,1000)
len(Normal)

1000

In [10]:
Viral_Pneumonia = list(data_dir.glob('Viral Pneumonia/*'))
Viral_Pneumonia = random.sample(Viral_Pneumonia,1000)
len(Viral_Pneumonia)

1000

In [11]:
images_dict = {
    'COVID': COVID,
    'Lung_Opacity': Lung_Opacity,
    'Normal': Normal,
    'Viral_Pneumonia': Viral_Pneumonia,
    
}

In [12]:
labels_dict = {
    'COVID': 0,
    'Lung_Opacity':1,
    'Normal': 2,
    'Viral_Pneumonia': 3,
    
}

In [14]:
import numpy as np
X, y = [], []

for label, images in images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(224,224))
        #resized_img=resized_img.astype(np.uint8)
        #resized_img=resized_img/np.max(resized_img)
        X.append(resized_img)
        y.append(labels_dict[label])

In [15]:
import numpy 
X = numpy.array(X)
y = numpy.array(y)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15)

In [17]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [18]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [19]:
import tensorflow as tf
num_of_flowers = 4

model = tf.keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(num_of_flowers)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 4)                 5124      
Total params: 2,263,108
Trainable params: 5,124
Non-trainable params: 2,257,984
_________________________________________________________________


In [20]:
model.compile(
  optimizer="adam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

model.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
100/100 [==============================] - 81s 751ms/step - loss: 0.7229 - acc: 0.7138
Epoch 2/5
100/100 [==============================] - 74s 744ms/step - loss: 0.4562 - acc: 0.8334
Epoch 3/5
100/100 [==============================] - 75s 747ms/step - loss: 0.3791 - acc: 0.8591
Epoch 4/5
100/100 [==============================] - 74s 743ms/step - loss: 0.3324 - acc: 0.8831
Epoch 5/5
100/100 [==============================] - 75s 746ms/step - loss: 0.3062 - acc: 0.8947


In [21]:
model.evaluate(X_test_scaled,y_test)

25/25 [==============================] - 19s 729ms/step - loss: 0.3357 - acc: 0.8675


[0.3356933891773224, 0.8675000071525574]

In [22]:
model.fit(X_train_scaled, y_train, epochs=10)

Epoch 1/10
100/100 [==============================] - 74s 736ms/step - loss: 0.2826 - acc: 0.9038
Epoch 2/10
100/100 [==============================] - 73s 729ms/step - loss: 0.2653 - acc: 0.9038
Epoch 3/10
100/100 [==============================] - 68s 684ms/step - loss: 0.2515 - acc: 0.9137
Epoch 4/10
100/100 [==============================] - 75s 752ms/step - loss: 0.2280 - acc: 0.9287
Epoch 5/10
100/100 [==============================] - 77s 767ms/step - loss: 0.2195 - acc: 0.9259
Epoch 6/10
100/100 [==============================] - 76s 764ms/step - loss: 0.2047 - acc: 0.9375
Epoch 7/10
100/100 [==============================] - 76s 760ms/step - loss: 0.2047 - acc: 0.9359
Epoch 8/10
100/100 [==============================] - 76s 761ms/step - loss: 0.1853 - acc: 0.9441
Epoch 9/10
100/100 [==============================] - 76s 758ms/step - loss: 0.1800 - acc: 0.9466
Epoch 10/10
100/100 [==============================] - 61s 610ms/step - loss: 0.1718 - acc: 0.9513


In [23]:
model.evaluate(X_test_scaled,y_test)

25/25 [==============================] - 11s 418ms/step - loss: 0.2849 - acc: 0.8975


[0.28489941358566284, 0.8974999785423279]